In [1]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

sys.path.append("../")
from pyfiles.processsound import trim_audio_and_save

sys.path.append("./../../BigVGAN/")
import bigvgan
from meldataset import get_mel_spectrogram

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
# feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
feat_base_dir = "/mntcephfs/data/audiow/shoinoue/Dataset/LibriTTS_R/features/"
fs = 16000

speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

# modelpath = "/mntcephfs/data/audiow/shoinoue/Model/hf_hub/bigvgan/models--nvidia--bigvgan_v2_24khz_100band_256x/snapshots/61df17db326f0876b7201d7a56c831898c836ef4"
modelpath = "/mntcephfs/data/audiow/shoinoue/Model/hf_hub/bigvgan/models--nvidia--bigvgan_v2_22khz_80band_fmax8k_256x/snapshots/189a02ed3b7957e8534b40e6314262df53536ece"
model = bigvgan.BigVGAN.from_pretrained(modelpath, use_cuda_kernel=False)
model.remove_weight_norm()
model = model.eval()

Loading config.json from local directory
Loading weights from local directory
Removing weight norm...


In [45]:
save = True
tempfile = "temp3.wav"
trim_threshold_in_db = 30

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
    wav, sr = librosa.load(tempfile, sr=model.h.sampling_rate, mono=True) 
    wav = torch.FloatTensor(wav).unsqueeze(0)
    mel = get_mel_spectrogram(wav, model.h)
    mel = np.array(mel[0])
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + "_80mel.npy"
    if save:
        np.save(savepath, mel)

  8%|▊         | 12149/160267 [13:56<3:37:05, 11.37it/s] 

[WARNING] Min value of input waveform signal is -1.0520296096801758


  9%|▊         | 13659/160267 [15:21<1:35:07, 25.69it/s]

[WARNING] Min value of input waveform signal is -1.0015461444854736


 10%|▉         | 15366/160267 [16:44<4:46:02,  8.44it/s]

...

 81%|████████  | 129400/160267 [1:43:47<24:34, 20.93it/s]  

[WARNING] Min value of input waveform signal is -1.0316983461380005


 96%|█████████▌| 153970/160267 [2:01:08<04:31, 23.17it/s]

[WARNING] Min value of input waveform signal is -1.0617516040802002


100%|██████████| 160267/160267 [2:05:13<00:00, 21.33it/s]


# Normalize

In [46]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [47]:
scaler_name = "LibriTTS-R_80mel"

In [48]:
scaler = StandardScaler()
files = []
for spk in tqdm(speakers):
    files += glob.glob(feat_base_dir + f"train*/{spk}/*/*_80mel.npy")
files.sort()
for path in tqdm(files):
    if "km500" in path:
        continue
    mel = np.load(path)
    scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)

100%|██████████| 149694/149694 [01:28<00:00, 1699.54it/s]


['ckpts/scalers/LibriTTS-R_80mel.save']